In [17]:
import pandas as pd
import os
import random
import torch
import yaml

In [18]:
random.seed(2024)

# Load original dataset

### Selecting datasets

In [19]:
root = '/root/dataset/ml-1m/'
output_path = './ml-1m-seq/'
dataset_name_list = [
    "ml-1m",
]
output_dataset_name_list = [
    "ml-1m",
]
user_threshold = 5
item_threshold = 5
max_seq_len = 200

### Load

In [20]:
dataset_list = []
for idx, dataset_name in enumerate(dataset_name_list):
    path = root + dataset_name + '.dat'
    dataset = pd.read_csv(path, header=None, sep='::')
    dataset.columns = ['user_id', 'item_id', 'rating', 'timestamp']
    dataset = dataset[dataset.rating > 3]
    dataset['domain'] = idx
    dataset_list.append(dataset)

/tmp/ipykernel_1409/684885453.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dataset = pd.read_csv(path, header=None, sep='::')


# Filter

### filter by number of interactions

In [21]:
filtered_dataset_list = []
for dataset in dataset_list:
    filtered_dataset = dataset.copy()
    while(True):
        ori_len = len(filtered_dataset)
        filtered_dataset = filtered_dataset[filtered_dataset['user_id'].map(filtered_dataset['user_id'].value_counts()) >= user_threshold]
        filtered_dataset = filtered_dataset[filtered_dataset['item_id'].map(filtered_dataset['item_id'].value_counts()) >= item_threshold]
        if len(filtered_dataset) == ori_len:
            break
    print('done!')
    filtered_dataset_list.append(filtered_dataset)

done!


# Map all ids

In [22]:
all_filtered_dataset = pd.concat(filtered_dataset_list)
all_user = all_filtered_dataset.user_id
all_item = all_filtered_dataset.item_id

In [23]:
user_id, user_token = pd.factorize(all_user)
item_id, item_token = pd.factorize(all_item)
num_users = len(user_token) + 1 # 0 id is for PAD
num_items = len(item_token) + 1 # 0 id is for PAD
user_mapping_dict = {_: idx + 1 for idx, _ in enumerate(user_token)} # 0 id is for PAD
item_mapping_dict = {_: idx + 1 for idx, _ in enumerate(item_token)} # 0 id is for PAD
print(user_token.shape)
print(item_token.shape)

(6034,)
(3125,)


In [24]:
all_filtered_dataset['user_id'] = all_filtered_dataset['user_id'].apply(lambda x: user_mapping_dict[x])
all_filtered_dataset['item_id'] = all_filtered_dataset['item_id'].apply(lambda x: item_mapping_dict[x])

In [25]:
mapped_dataset_list = [all_filtered_dataset[all_filtered_dataset['domain'] == idx] for idx in range(len(filtered_dataset_list))]

# Generate sequences

### Define padding token id

In [26]:
PAD = 0

### In-domain sequences

In [27]:
def truncate_or_pad(seq):
    cur_seq_len = len(seq)
    if cur_seq_len > max_seq_len:
        return seq[-max_seq_len:], max_seq_len
    else:
        return seq + [PAD] * (max_seq_len - cur_seq_len), cur_seq_len

In [ ]:
def to_list(x):
    return list(x)[:-2]
dataset = mapped_dataset_list[0]
dataset = dataset.sort_values(by=['user_id', 'timestamp'])
user_group = dataset.groupby('user_id')['item_id'].apply(to_list)
torch.save(user_group.tolist(), 'pattern-ml-1m.pth')

In [12]:

for idx, dataset in enumerate(mapped_dataset_list):
    dataset = dataset.sort_values(by=['user_id', 'timestamp'])
    user_group = dataset.groupby('user_id')['item_id'].apply(list)
    train, val, test = [], [], []
    for user_id, user_seq in list(zip(user_group.index, user_group.tolist())):
        # user_seq = user_seq[-max_seq_len:]
        # iterate on each user sequence
        # ------ test sample ------------
        history, seq_len = truncate_or_pad(user_seq[:-1])
        target_data = user_seq[-1]
        label = 1
        domain_id = [idx] * max_seq_len
        user_hist = user_seq[:-1]
        test.append([user_id, history, target_data, seq_len, label, domain_id, user_hist])
        # ------ val sample -------------
        history, seq_len = truncate_or_pad(user_seq[:-2])
        target_data = user_seq[-2]
        label = 1
        domain_id = [idx] * max_seq_len
        user_hist = user_seq[:-2]
        val.append([user_id, history, target_data, seq_len, label, domain_id, user_hist])
        # ------ train sample -----------
        train_seq = user_seq[:-2]
        assert len(train_seq) >=2
        s, seq_len = 0, 1
        while(True):
            if s + seq_len == len(train_seq):
                break
            if seq_len > max_seq_len:
                s += 1
                seq_len = max_seq_len
            history, _ = truncate_or_pad(train_seq[s:s + seq_len])
            target_data = train_seq[s + seq_len]
            label = 1
            domain_id = idx
            train.append([user_id, history, target_data, seq_len, label, domain_id])

            seq_len += 1
    torch.save(train, os.path.join(output_path, output_dataset_name_list[idx], 'train.pth'))
    torch.save(val, os.path.join(output_path, output_dataset_name_list[idx], 'val.pth'))
    torch.save(test, os.path.join(output_path, output_dataset_name_list[idx], 'test.pth'))
    print('{} done!'.format(output_dataset_name_list[idx]))

ml-1m done!


In [22]:


def truncate_or_pad(seq):
    cur_seq_len = len(seq)
    if cur_seq_len > max_seq_len:
        return seq[-max_seq_len:], max_seq_len
    else:
        return seq + [PAD] * (max_seq_len - cur_seq_len), cur_seq_len

for idx, dataset in enumerate(mapped_dataset_list):
    dataset = dataset.sort_values(by=['user_id', 'timestamp'])
    user_group = dataset.groupby('user_id')['item_id'].apply(list)
    train, val, test = [], [], []
    for user_id, user_seq in list(zip(user_group.index, user_group.tolist())):
        user_seq = user_seq[-max_seq_len:]
        # iterate on each user sequence
        # ------ test sample ------------
        history, seq_len = truncate_or_pad(user_seq[:-1])
        target_data = user_seq[-1]
        label = 1
        domain_id = [idx] * max_seq_len
        test.append([user_id, history, target_data, seq_len, label, domain_id])
        # ------ val sample -------------
        history, seq_len = truncate_or_pad(user_seq[:-2])
        target_data = user_seq[-2]
        label = 1
        domain_id = [idx] * max_seq_len
        val.append([user_id, history, target_data, seq_len, label, domain_id])
        # ------ train sample -----------
        history, seq_len = truncate_or_pad(user_seq[:-3])
        target_data, _ = truncate_or_pad(user_seq[-seq_len-2:-2])
        label = [1] * seq_len + [PAD] * (max_seq_len - seq_len)
        domain_id = [idx] * max_seq_len
        train.append([user_id, history, target_data, seq_len, label, domain_id])
    torch.save(train, os.path.join(output_path, output_dataset_name_list[idx], 'train.pth'))
    torch.save(val, os.path.join(output_path, output_dataset_name_list[idx], 'val.pth'))
    torch.save(test, os.path.join(output_path, output_dataset_name_list[idx], 'test.pth'))
    print('{} done!'.format(output_dataset_name_list[idx]))

ml-1m done!


### mixed sequence

# Save

In [15]:
for d_name, d in zip(output_dataset_name_list, mapped_dataset_list):
    d.to_csv(os.path.join(output_path, d_name, 'inter.csv'), sep=',', index=None)

In [16]:
print(mapped_dataset_list[0]['user_id'].unique().shape)
print(mapped_dataset_list[0]['item_id'].unique().shape)

(6034,)
(3125,)
